**Loading and Preprocessing Data**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model

def load_data(file_path):
    df_credit = pd.read_csv(file_path)
    scaler = StandardScaler()
    df_credit['Amount'] = scaler.fit_transform(df_credit[['Amount']])
    df_credit.drop(['Time'], axis=1, inplace=True)
    return df_credit

data_path_credit = 'credit_card.csv'
df_credit = load_data(data_path_credit)
X_train_credit, X_test_credit = train_test_split(df_credit, test_size=0.2, random_state=42)


**Creating a Masked Dataset**

In [ ]:
def geometric_mask(input_data, mask_rate=0.1):
    mask = np.random.geometric(p=mask_rate, size=input_data.shape) <= 1
    return input_data * mask

X_train_masked_custom = geometric_mask(X_train_credit.values)


**Building the Autoencoder Model**

In [ ]:

def custom_contrastive_loss(y_true_custom, y_pred_custom, margin_custom=1):
    return tf.reduce_mean(y_true_custom * tf.square(y_pred_custom) + (1 - y_true_custom) * tf.square(tf.maximum(margin_custom - y_pred_custom, 0)))

def custom_build_autoencoder(input_dim_custom):
    inputs_custom = Input(shape=(input_dim_custom,))
    encoded_custom = Dense(64, activation='relu')(inputs_custom)
    decoded_custom = Dense(input_dim_custom, activation='sigmoid')(encoded_custom)
    model_custom = Model(inputs=inputs_custom, outputs=decoded_custom)
    model_custom.compile(optimizer='adam', loss=custom_contrastive_loss)
    return model_custom

def custom_build_discriminator(input_dim_custom):
    inputs_custom = Input(shape=(input_dim_custom,))
    x_custom = Dense(64, activation='relu')(inputs_custom)
    x_custom = Dense(64, activation='relu')(x_custom)
    outputs_custom = Dense(1, activation='sigmoid')(x_custom)
    model_custom = Model(inputs=inputs_custom, outputs=outputs_custom)
    model_custom.compile(optimizer='


**Building the Discriminator Model**

In [ ]:
def train_custom_autoencoder(custom_model, custom_data, epochs=10, batch_size=32):
    for epoch in range(epochs):
        np.random.shuffle(custom_data)
        for i in range(0, len(custom_data), batch_size):
            batch_custom_data = custom_data[i:i+batch_size]
            if batch_custom_data.shape[0] != batch_size:
                continue
            loss_custom = custom_model.train_on_batch(batch_custom_data, batch_custom_data)
            print(f"Epoch {epoch + 1}, Batch {i // batch_size + 1}, Loss: {loss_custom}")

train_custom_autoencoder(autoencoder_custom, X_train_masked_custom, epochs=3, batch_size=5000)


Epoch 1, Batch 1, Loss: 0.36111724376678467
Epoch 1, Batch 2, Loss: 0.355888694524765
Epoch 1, Batch 3, Loss: 0.35830187797546387
Epoch 1, Batch 4, Loss: 0.3594941794872284
Epoch 1, Batch 5, Loss: 0.3566878139972687
Epoch 1, Batch 6, Loss: 0.3573008179664612
Epoch 1, Batch 7, Loss: 0.3553857207298279
Epoch 1, Batch 8, Loss: 0.35467275977134705
Epoch 1, Batch 9, Loss: 0.35301852226257324
Epoch 1, Batch 10, Loss: 0.3521229326725006
Epoch 1, Batch 11, Loss: 0.350278377532959
Epoch 1, Batch 12, Loss: 0.3494091033935547
Epoch 1, Batch 13, Loss: 0.34853890538215637
Epoch 1, Batch 14, Loss: 0.34775251150131226
Epoch 1, Batch 15, Loss: 0.3462434709072113
Epoch 1, Batch 16, Loss: 0.3448834717273712
Epoch 1, Batch 17, Loss: 0.34428098797798157
Epoch 1, Batch 18, Loss: 0.34304279088974
Epoch 1, Batch 19, Loss: 0.34210482239723206
Epoch 1, Batch 20, Loss: 0.3409128487110138
Epoch 1, Batch 21, Loss: 0.3406008780002594
Epoch 1, Batch 22, Loss: 0.3397917151451111
Epoch 1, Batch 23, Loss: 0.3391832113

**Training the Autoencoder**

In [ ]:
def train_custom_discriminator(custom_discriminator, custom_autoencoder, custom_real_data, epochs=10, batch_size=32):
    for epoch in range(epochs):
        np.random.shuffle(custom_real_data)
        for i in range(0, len(custom_real_data), batch_size):
            custom_real_batch = custom_real_data[i:i+batch_size]
            if custom_real_batch.shape[0] != batch_size:
                continue
            custom_fake_batch = custom_autoencoder.predict(custom_real_batch)
            custom_x_combined_batch = np.concatenate((custom_real_batch, custom_fake_batch))
            custom_y_combined_batch = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))))
            loss_custom = custom_discriminator.train_on_batch(custom_x_combined_batch, custom_y_combined_batch)
            print(f"Epoch {epoch + 1}, Discriminator Batch {i // batch_size + 1}, Loss: {loss_custom}")

train_custom_discriminator(discriminator_custom, autoencoder_custom, X_train_credit.values, epochs=3, batch_size=5000)



157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Epoch 1, Discriminator Batch 1, Loss: 0.6853143572807312
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Epoch 1, Discriminator Batch 2, Loss: 0.6725725531578064
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Epoch 1, Discriminator Batch 3, Loss: 0.660690188407898
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Epoch 1, Discriminator Batch 4, Loss: 0.6495654582977295
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Epoch 1, Discriminator Batch 5, Loss: 0.6388702988624573
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Epoch 1, Discriminator Batch 6, Loss: 0.6288747191429138
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Epoch 1, Discriminator Batch 7, Loss: 0.6190589070320129
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Epoch 1, Discriminator Batch 8, Loss: 0.6096349954605103
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Epoch 1, Discriminator Batch 9, Loss: 0.6006031632423401
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Epoch 1, Discriminator Batch 10, Loss: 0.5919048190116882
157/157 ━━━━━━━━━━━━

**Training the Discriminator**

In [ ]:
def detect_custom_anomalies(custom_model, custom_data):
    custom_predictions = custom_model.predict(custom_data)
    custom_mse = np.mean(np.power(custom_data - custom_predictions, 2), axis=1)
    return custom_mse

anomaly_scores_custom = detect_custom_anomalies(autoencoder_custom, X_test_credit.values)



1781/1781 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step


**Detecting Anomalies and Setting Thresholds**

In [ ]:
# Calculate a threshold
custom_mse_threshold = np.mean(anomaly_scores_custom) + 2 * np.std(anomaly_scores_custom)
print("Custom MSE Threshold: ", custom_mse_threshold)

# Identify anomalies
custom_anomalies = anomaly_scores_custom > custom_mse_threshold

# Show anomalous data
print("Anomaly Detected:")
print(X_test_credit[custom_anomalies])

# Optional: Count of anomalies
print("Number of anomalies detected:", np.sum(custom_anomalies))



MSE Threshold:  8.362404027002002
Anomaly Detected:
               V1         V2         V3        V4         V5         V6  \
43428  -16.526507   8.584972 -18.649853  9.505594 -13.793819  -2.832404   
219257 -29.942972 -25.831782 -16.227512  6.690679 -20.787846  13.085694   
113740  -5.136552   5.746647  -3.838599 -0.329163   1.288327   0.251632   
154372  -8.339783   7.278206  -6.017969 -2.170078  -2.006120  -1.717894   
172250  -5.013928  -3.169697  -3.283770  1.125001  -9.982772   7.355276   
...           ...        ...        ...       ...        ...        ...   
123326 -10.924306   8.486126  -5.303020 -2.547358  -5.055657  -1.375718   
92366   -8.254743   6.837986  -6.780968  0.670752  -4.482530  -1.176162   
33756   -7.504323 -15.633772  -4.215051  1.507090  -5.412604   3.243205   
44614   -9.482402   5.448324  -7.587924  1.334433  -5.916629  -1.842644   
248050  -2.897780  -9.166194  -5.001077  0.732565  -1.804056   1.973025   

               V7        V8        V9        V1